In [1]:
import pyaudio
import numpy as np
import wave
import keyboard
import time

#set 's' as the start key 
Start_key='s'

background=0

#record the background
def record_background():
    time.sleep(1)
    CHUNK = 1600
    resolution = pyaudio.paInt16 
    CHANNELS = 1 
    sampling_rate = 16000
    record_time= 1
    Expected_num_of_chunks= int(sampling_rate/CHUNK*record_time) 
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    data = stream.read(10)
    np_array=np.array(data)
        
    #convert hexadecimal numbers to decimal ones
    int_np_array=np.frombuffer(np_array, np.int16)
    int_64 = np.array(int_np_array,dtype='int64')
    energy=10*np.log10((np.square(int_64)))
    background=np.mean(energy)

    #stop recording
    stream.stop_stream()
    stream.close()
    p.terminate()
    return background

def record(background):
    #initialize parameters for recording with pyaudio
    CHUNK = 1600
    resolution = pyaudio.paInt16
    CHANNELS = 1
    sampling_rate = 16000
    
    #initialize parameters for classifying frames
    forgetfactor = 1
    isSpeech = False
    level = 0
    adjustment = 0.05
    threshold = 1.2
    init = 0
    count = 0
    frame_num = 0
    
    p=pyaudio.PyAudio()
    stream = p.open(format=resolution, channels=CHANNELS, rate=sampling_rate, input=True, frames_per_buffer=CHUNK)
    frames=[]
    intframes=[]
    while True:
        #get data
        data = stream.read(10)
        frames.append(data)
        np_array=np.array(data)
        
        #convert hexadecimal numbers to decimal ones
        int_np_array=np.frombuffer(np_array, np.int16)
        for i in int_np_array:
            intframes.append(i)
        int_64 = np.array(int_np_array,dtype='int64')
        energy=10*np.log10((np.square(int_64)))
        
        #initialize background and level
        if init==0:
            init+=1
            level=energy[0]
        energy=np.mean(energy)

        #adaptive endpointing algo
        current=energy
        level=((level*forgetfactor)+current)/(1+forgetfactor)
        if(current<background):
            background=current
        else:
            background+=(current-background)*adjustment
        if(level<background): level=background
        if(level-background>threshold):
            count=0
            isSpeech = True
        else: count+=1
        if count>5: isSpeech=False
        print((level,background,level-background))
        
        #recording end condition
        if not isSpeech and init==1:
            print("stop recording")
            stream.stop_stream()
            stream.close()
            p.terminate()
            break
    return intframes

def listen():
    intframes=[]
    while True:
        if keyboard.is_pressed(Start_key):
            print('start')
            background=record_background()
            intframes=record(background)
            break
    return intframes

In [3]:
intframes=listen()

start


C:\Users\Song Eugene\AppData\Local\Temp\ipykernel_28356\2646598895.py:29: RuntimeWarning: divide by zero encountered in log10
  energy=10*np.log10((np.square(int_64)))


(-inf, nan, nan)
stop recording


C:\Users\Song Eugene\AppData\Local\Temp\ipykernel_28356\2646598895.py:70: RuntimeWarning: divide by zero encountered in log10
  energy=10*np.log10((np.square(int_64)))
C:\Users\Song Eugene\AppData\Local\Temp\ipykernel_28356\2646598895.py:84: RuntimeWarning: invalid value encountered in scalar subtract
  background+=(current-background)*adjustment


In [4]:
print(intframes)
#print(10*np.log(np.sum(np.square(intframes[-1]))))

[0, 0, -1, 0, 0, 0, 0, -1, 0, 0]


In [5]:
import IPython.display as ipd
ipd.Audio(intframes,rate=16000)